In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.2 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import transformers

print(transformers.__version__)

4.27.3


## Preparing the dataset

In [4]:
from datasets import load_dataset
datasets = load_dataset('Sree1994/blm_strict_small')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating valid split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Sree1994___parquet/Sree1994--blm_strict_small-3934cc01075622aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets["train"][10]

{'text': '\tThe late Ricky Wilson of The B-52\'s notes his love of Hot Tamales in their 1983 single "Song for a Future Generation".\n'}

## Causal Language modeling

In [6]:
model_checkpoint = "EleutherAI/gpt-neo-1.3B"

In [7]:
from transformers import AutoTokenizer
import torch

    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [10]:
tokenized_datasets["train"][1]

{'input_ids': [198], 'attention_mask': [1]}

In [11]:
# block_size = tokenizer.model_max_length
block_size = 128

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [14]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

" century, along with the Portuguese adventurer Diogo Veloso from Amarante, Blas Ruiz was the first European to ever set foot in Laos.\n= = = Makeup brush = = =\n\tLongstreet's attack commenced with a 30-minute artillery barrage by 36 guns that was particularly punishing to the Union infantry in the Peach Orchard and the troops and batteries on Houck's Ridge. Maj. Gen. John Bell Hood's division deployed in Biesecker's Woods on Warfield Ridge (the southern extension of Seminary Ridge) in two lines of two brigades each: at the left front, Brig."

In [16]:
from transformers import AutoModelForCausalLM, AutoConfig
config = AutoConfig.from_pretrained(
    model_checkpoint,
    vocab_size=len(tokenizer),
    random_init=True
)
model = AutoModelForCausalLM.from_config(config)
model.init_weights()

In [ ]:
#from transformers import AutoModelForCausalLM
#model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [17]:
from transformers import Trainer, TrainingArguments

We pass along all of those to the `Trainer` class:

In [18]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-babylm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs=10,
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["valid"],
)

And we can train our model:

In [20]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,7.250700,7.132274
2,6.886400,6.904308
3,6.669900,6.789542
4,6.475200,6.725535


RuntimeError: ignored

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 697
  Batch size = 8


Perplexity: 18.89
